# 继承Moudle类来构造模型

Moudle类是nn模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的结构。需要重载Moudle类的__init__函数及forward函数。分别是用于创建模型参数和定义前向计算。

In [82]:
import torch
from torch import nn 

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output=nn.Linear(256,10)

    def forward(self,x):
        a=self.act(self.hidden(x))
        return self.output(a)


实例化MLP类得到模型变量net

In [83]:
net=MLP()
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


输入数据X做一次前向计算。其中，net(X)会调用MLP继承自Module类的__call__函数，这个函数将调用MLP类定义的forward函数来完成前向计算。

In [84]:
net(torch.rand(2,784))

tensor([[-0.0123, -0.1601, -0.1327, -0.1055, -0.0150, -0.0542, -0.0173,  0.0336,
          0.1206,  0.1027],
        [-0.0837, -0.0960, -0.1623, -0.1244,  0.0833, -0.0436,  0.0164, -0.0550,
          0.1342,  0.0580]], grad_fn=<AddmmBackward>)

这里并没有将Moudle类命名为Layer或model之类的名字，因为该类是一个可供选择自由组建的部件。它的子类既可以是一个层，又可以是一个模型，或者是模型的一部分。

# Module的子类

PyTorch还实现了继承自Module的可以方便构建模型的类: 如Sequential、ModuleList和ModuleDict等等。

* Sequential类

当模型的前向计算为简单串联各个层的计算时，Sequential类可以通过更简单的方式定义模型。  
他可以接受一个子模块的有序序列(OrderDict)或者一系列子模块作为参数来逐一添加Module的实例，而模型的前向计算就是按照这些实例的添加顺序逐一计算。

自己实现Sequential类

In [85]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super().__init__()
        if len(args)==1 and isinstance(args[0],OrderedDict):
            for key,module in args[0].items():
                self.add_module(key,module)
        else:
            for index,module in enumerate(args):
                self.add_module(str(index),module)
    def forward(self,input):
        for module in self._modules.values():
            input = module(input)
        return input

用MySequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算

In [86]:
net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10)
)
print(net)
net(torch.rand(2,784))

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.2222, -0.2102, -0.0335,  0.1461,  0.0089,  0.0768, -0.0336, -0.0614,
         -0.0328,  0.1080],
        [ 0.3054, -0.0729, -0.0752,  0.1059,  0.0129,  0.0121,  0.1086, -0.0806,
         -0.0863,  0.0763]], grad_fn=<AddmmBackward>)

* ModuleList类

ModuleList接受一个子模块列表作为输入，然后也可以类似List呢样进行append和extend操作

In [87]:
net = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
#append
net.append(nn.Linear(256,10))
#索引访问
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


Sequential和ModuleList都可以模块化构造网络。  
区别是：  
ModuleList仅仅是一个存储各种模块的列表，这些模块间没有联系也没有顺序（所以不用保证相邻层的输入输出匹配），而且没有实现forward方法。  
Sequential内的模块会按照顺序排列组合，要保证相邻层的输入输出大小匹配，内部自动实现forward方法

ModuleList的出现只是让网络的前向传播时更加灵活

In [88]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.linears=nn.ModuleList([nn.Linear(10,10) for i in range(10)])
    def forward(self,x):
        for i,l in enumerate(self.linears):
            x=self.linears[i//2](x)+l(x)
        return x

In [89]:
print(MyModule())

MyModule(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
  )
)


如上所示，ModuleList不同于一般的Python的List，加入到Modulist里面的所有的模块的参数会自动添加到整个网络中。

In [90]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super().__init__()
        self.linears=nn.ModuleList([nn.Linear(10,10)])
class Module_List(nn.Module):
    def __init__(self):
        super().__init__()
        self.linears=[nn.Linear(10,10)]
print(Module_ModuleList())
print(Module_List())

Module_ModuleList(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
)
Module_List()


* ModuleDict类

ModuleDict接受一个子模块的字典作为输入，然后也可以类似字典呢样进行添加访问操作

In [91]:
net=nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU()
})
net['output']=nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数，同样ModuleDict里的所有模块的参数会被自动添加到整个网路

# 构造复杂的网络

虽然上面介绍的类可以使得模型的构造更加灵活，且不需要forward函数，但是直接继承Module类可以极大的拓展模型构造的灵活性。  
我们通过get_constant函数创建训练值不被迭代的参数，即常数参数。  
在前向计算中，除了使用创建的常数参数外，我们还要使用Tenso的函数和Python的控制流，并多次调用相同的层。

In [92]:
class FancyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.randWeight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)

    def forward(self,x):
        x=self.linear(x)
        #使用创建的常参数
        x=nn.functional.relu(torch.mm(x,self.randWeight)+1)

        #复用全连接层
        x=self.linear(x)

        #控制流
        while x.norm().item() > 1 :
            x /= 2
        if x.norm().item() < 0.8 :
            x*=10
        return x.sum()
x=torch.rand(2,20)
net = FancyMLP()
print(net)
net(x)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-6.8642, grad_fn=<SumBackward0>)

In [93]:
class NestMlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(40,30),nn.ReLU())
    def forward(self,x):
        return self.net(x)

In [94]:
net = nn.Sequential(NestMlp(),nn.Linear(30,20),FancyMLP())
print(net)

Sequential(
  (0): NestMlp(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)
